# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
## uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

Below, we define the connection to a Cassandra instance with the syntax we have learnt in the lessons of this course.

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

We create the keyspace that we will need for this project. Since this is the first project of the nanodegree program, it is named "project1".

In [6]:
# Create a Keyspace

# Exception handling for the creation of the keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

Now, we connect to the keyspace that we have just created.

In [7]:
# Set KEYSPACE to the keyspace specified above

# Exception handling for the connection to the keyspace
try:
    session.set_keyspace('project1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### CREATE statement 

We create the structure of the table for the first query. Since we need to get specific data for a certain song of a given session, we will use sessionId and itemInSession as the columns for our composite key.

We include also the columns we need to retrieve with the query afterwards: artist, song and length of the song.

In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_data_item_in_session"
query = query + "(sessionId int, \
    itemInSession int, \
    artist text, \
    song text, \
    length float, \
    PRIMARY KEY (sessionId, itemInSession))"

# Exception handling for the execution of the query 
try:
    session.execute(query)
except Exception as e:
    print(e)

#### INSERT statement

We populate our table with data from the csv "event_datafile_new".

The columns we need in this case (and their index) are:
- 8: sessionId (converted to integer)
- 3: itemInSession (converted to integer)
- 0: artist
- 9: song
- 5: song's length (converted to float)

In [9]:
# We open the csv file and insert the appropriate columns into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_data_item_in_session(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement

We retrieve the requested data (artist, song and length) for the specific item = 4 of the session with id = 338.

In [10]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length \
    FROM song_data_item_in_session \
    WHERE sessionId = 338 AND itemInSession = 4"

# Exception handling for the execution of the query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Print the data obtained with the query
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### CREATE statement 

We create the structure of the table for the second query. In this case, we have to get all the artists and songs for a certain user in a given session. Besides, we are asked to get the results sorted by itemInSession. Thus, we will define our composite partition key with the values of userId and sessionId, and our table will have itemInSession as clustering column.

We include also the columns we need to retrieve with the query afterwards: artist, song and user (defined with first and last name).

In [11]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_user_data_session"
query = query + "(userId int, \
    sessionId int, \
    itemInSession int, \
    artist text, \
    song text, \
    firstName text, \
    lastName text, \
    PRIMARY KEY ((userId, sessionId), itemInSession))"

# Exception handling for the execution of the query 
try:
    session.execute(query)
except Exception as e:
    print(e)         

#### INSERT statement

We populate our table with data from the csv "event_datafile_new".

The columns we need in this case (and their index) are:
- 10: userId (converted to integer)
- 8: sessionId (converted to integer)
- 3: itemInSession (converted to integer)
- 0: artist
- 9: song
- 1: user's first name
- 4: user's last name

In [12]:
# We open the csv file and insert the appropriate columns into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_data_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT statement

We retrieve the requested data (artist, song and user) for the userId = 10 and the session with id = 182.

In [13]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName \
    FROM song_user_data_session \
    WHERE userId = 10 AND sessionId = 182 \
    ORDER BY itemInSession"

# Exception handling for the execution of the query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Print the data obtained with the query
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### CREATE statement 

We create the structure of the table for this last query. We have to get every user name, identified with first and last name, who have listened to a specific song "All Hands Against His Own".

Since we have to identify unique users and the combination of first + last name could have duplicates, we include userId in our primary key. As we will need the song name in the where condition, we also include this value. 

We add the columns we are asked to get afterwards: first and last name of the user.

In [14]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_data_given_song"
query = query + "(song text, \
    userId int, \
    firstName text, \
    lastName text, \
    PRIMARY KEY (song, userId))"

# Exception handling for the execution of the query
try:
    session.execute(query)
except Exception as e:
    print(e)             

#### INSERT statement

We populate our table with data from the csv "event_datafile_new".

The columns we need in this case (and their index) are:
- 9: song
- 10: userId (converted to integer)
- 1: user's first name
- 4: user's last name

In [15]:
# We open the csv file and insert the appropriate columns into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_data_given_song(song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement

We retrieve the requested data (first and last name of user) for every one who have listened to the song "All Hands Against His Own".

In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName \
    FROM user_data_given_song \
    WHERE song = 'All Hands Against His Own'"

# Exception handling for the execution of the query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print the data obtained with the query  
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop first table
query = "DROP TABLE song_data_item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Drop second table
query = "DROP TABLE song_user_data_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Drop third table
query = "DROP TABLE user_data_given_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()